## Textdata anlaysis 실행파일

In [1]:
from textanal import contents_clean, contents_separation, distribution
from textanal.morphanal import analyze_corpus
from textanal.TFIDF import TF_IDF
from textanal.cooccurence import coocur
import pandas as pd
from tqdm import tqdm
import pickle

### Text 전처리

In [2]:
df = pd.read_csv("a.csv")
df['clean'] = df['본문'].apply(lambda x : contents_clean(x)) # 불용어 제거 함수
df['sep'] = df['clean'].apply(lambda x : contents_separation(x)) # contents를 sentence 단위로 분할하는 함수

In [ ]:
df2 = pd.DataFrame(columns = ['str', 'idx']) # dataframe 만들어 문장 분리 실행
for idx, lst in tqdm(enumerate(df['sep'].tolist())):
    split_df= distribution(idx,lst) # sentence 단위로 분할해서 데이터 프레임으로 변환하는 함수
    df2 = pd.concat([df2,split_df])
else:
    # 정상적으로 완료될 경우 결측치 제거, 인덱스 초기화 진행.
    df2.dropna(axis=0, inplace=True)
    df2.reset_index(drop=True, inplace=True)
df2 # 분리된 문장을 담은 dataframe 출력

### 피클저장(나중에 지워야함)

In [8]:
# 데이터프레임을 피클 파일로 저장
with open('b.pkl', 'wb') as file:
    pickle.dump(df2, file)

In [ ]:
# 저장된 피클 파일을 불러오기
with open('b.pkl', 'rb') as file:
    df2 = pickle.load(file)
df2

### 형태소 분석

In [ ]:
# 명사, 동사, 형용서, 조사 빈도 데이터프레임들과 형태소 단위로 나눈 데이터프레임을 반환
noun_df, verb_df, adj_df, josa_df, morph_df = analyze_corpus(df2)

### TF-IDF(단어 빈도-역 문서 빈도, Term Frequency-Inverse Document Frequency) 출력

In [4]:
# TF-IDF 결과값 반환. only_kor=True일때는 한글로만 이루어진 TF-IDF데이터프레임 반환
word_count, tfidf_DF = TF_IDF(morph_df, josa_df, only_kor=True)

### 단어동시출현행렬(co-occurence matrix) 출력

In [ ]:
# 단어동시출현행렬 = 원본 dataframe, 상위50단어 dataframe, 공출현빈도행렬(1-mode matrix) 순으로 출력
freq_df2, freq_df3, word_co_matx = coocur(morph_df, noun_df)